In [6]:
import pandas as pd
import glob
import os
import numpy as np
import pickle
import statistics
import community
import matplotlib
# matplotlib.use("Qt5Agg")
import matplotlib.pyplot as plt

import networkx as nx

import analysis as an

from visbrain.objects import BrainObj, ColorbarObj, SceneObj, SourceObj
from visbrain.io import download_file, read_stc

/Users/gracer/.local/lib/python3.6/site-packages/vispy/visuals/isocurve.py:22: UserWarning: VisPy is not yet compatible with matplotlib 2.2+
  warnings.warn("VisPy is not yet compatible with matplotlib 2.2+")


In [4]:
basepath='/Users/gracer/Google Drive/HCP_graph/1200/datasets/'
summary_dict=an.onetoughjar(os.path.join(basepath,'tmp','summary_dict_11-14-2019_04-33-33'))
# dict_keys(['mean_FC', 'graphs', 'clustering_coeff', 'btn_centrality', 'PC', 'modules'])
# summary_dict['NR']['no']['modules']['partition']
summary_dict['NR']['no']['graphs'].nodes(data=True)

an.grace_graph(summary_dict['NR']['no']['graphs'], 'clustering', 'Normal weight', 1)
an.grace_graph(summary_dict['NR']['ov']['graphs'], 'clustering', 'Overweight', 1)
an.grace_graph(summary_dict['NR']['ob']['graphs'], 'clustering', 'Obese', 1)

for key, value in summary_dict.items():
    for k,v in value.items():
        # community.induced_graph(partition dictionary, graph)
         comm_graph = community.induced_graph(v['modules']['partition'], v['graphs'])
         v.update(comm_graph = comm_graph)

(100,)
(100,)
(100,)


In [5]:
an.module_fig(summary_dict['NR']['no']['comm_graph'], 'Normal')
an.module_fig(summary_dict['NR']['ov']['comm_graph'], 'Overweight')
an.module_fig(summary_dict['NR']['ob']['comm_graph'], 'Obese')

# Using visbrain to look at the cifti files

In [7]:
# Scene creation
sc = SceneObj(bgcolor='black', size=(1400, 1000))
# Colorbar default arguments. See `visbrain.objects.ColorbarObj`
CBAR_STATE = dict(cbtxtsz=12, txtsz=10., width=.1, cbtxtsh=3.,
                  rect=(-.3, -2., 1., 4.))
KW = dict(title_size=14., zoom=1.2)


Creation of a scene
11-15-2019_10-58-19 Creation of a scene 


In [8]:
# Download the annotation file of the left hemisphere lh.aparc.a2009s.annot
path_to_file1 = download_file('lh.aparc.a2009s.annot', astype='example_data')
# Define the brain object (now you should know how to do it)
b_obj_parl = BrainObj('inflated', hemisphere='left', translucent=False)
# Print parcellates included in the file
# print(b_obj_parl.get_parcellates(path_to_file1))
# Finally, parcellize the brain and add the brain to the scene
b_obj_parl.parcellize(path_to_file1)
sc.add_to_subplot(b_obj_parl, row=1, col=1, rotate='left',
                  title='Parcellize using the Desikan Atlas', **KW)


11-15-2019_10-58-20 Downloading /Users/gracer/visbrain_data/example_data/lh.aparc.a2009s.annot 
STATUS : 100.0% 1318912 / 1314060
BrainObj(name='inflated') created
11-15-2019_10-58-21 BrainObj(name='inflated') created 
    Annot file loaded (/Users/gracer/visbrain_data/example_data/lh.aparc.a2009s.annot)
11-15-2019_10-58-22     Annot file loaded (/Users/gracer/visbrain_data/example_data/lh.aparc.a2009s.annot) 
WARNING | left hemisphere(s) inferred from filename
11-15-2019_10-58-22 left hemisphere(s) inferred from filename 
    Select all parcellates
11-15-2019_10-58-22     Select all parcellates 
    Use default color included in the file
11-15-2019_10-58-22     Use default color included in the file 
    Search parcellates using labels
11-15-2019_10-58-22     Search parcellates using labels 
    Selected parcellates : G_and_S_frontomargin, G_and_S_occipital_inf, G_and_S_paracentral, G_and_S_subcentral, G_and_S_transv_frontopol, G_and_S_cingul-Ant, G_and_S_cingul-Mid-Ant, G_and_S_cingu

In [9]:
# Download the lh.sig.nii.gz file
file = download_file('lh.sig.nii.gz', astype='example_data')
# Define the [...] you know
b_obj_fmri = BrainObj('inflated', translucent=False, sulcus=True)
# Add fMRI activation and hide every activation that is under 5.
b_obj_fmri.add_activation(file=file, clim=(5., 20.), hide_under=5,
                          cmap='viridis', hemisphere='left')
sc.add_to_subplot(b_obj_fmri, row=2, col=1, title='Add fMRI activation',
                  rotate='left', **KW)


11-15-2019_10-58-22 Downloading /Users/gracer/visbrain_data/example_data/lh.sig.nii.gz 
STATUS : 100.0% 557056 / 555499
BrainObj(name='inflated') created
11-15-2019_10-58-23 BrainObj(name='inflated') created 
    Add overlay to the inflated brain template (/Users/gracer/visbrain_data/example_data/lh.sig.nii.gz)
11-15-2019_10-58-23     Add overlay to the inflated brain template (/Users/gracer/visbrain_data/example_data/lh.sig.nii.gz) 
    BrainObj(name='inflated') added to the scene
11-15-2019_10-58-23     BrainObj(name='inflated') added to the scene 


In [10]:
# Link the rotation of subplots (row=0, col=1) and (row=1, col=2)
# sc.link((0, 1), (1, 2))
# Screenshot of the scene
# sc.screenshot('ex_brain_obj.png', transparent=True)

sc.preview()

11-15-2019_10-58-28 No OpenGL_accelerate module loaded: No module named 'OpenGL_accelerate' 
